In [29]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch_geometric
import tensorflow as tf
import networkx as nx
import awkward as ak
import uproot

In [32]:
testfile = uproot.open('testdata.root')
trackster = testfile['ntuplizer/tracksters;4']
df = ak.to_pandas(trackster.arrays(entry_start=0, entry_stop=10))
event = df.loc[0]
df.head()

event  NClusters  NTracksters      time  \
entry subentry subsubentry                                            
0     0        0                1        417           24  0.003388   
               1                1        417           24  0.003388   
               2                1        417           24  0.003388   
               3                1        417           24  0.003388   
               4                1        417           24  0.003388   

                            timeError  regressed_energy  raw_energy  \
entry subentry subsubentry                                            
0     0        0             0.004523        129.730362   131.40152   
               1             0.004523        129.730362   131.40152   
               2             0.004523        129.730362   131.40152   
               3             0.004523        129.730362   131.40152   
               4             0.004523        129.730362   131.40152   

                            raw_em_energy     raw_pt  raw_em_pt  ...  \
entry subentry subsubentry                                       ...   
0     0        0                      0.0  21.647354        0.0  ...   
               1                      0.0  21.647354        0.0  ...   
               2                      0.0  21.647354        0.0  ...   
               3                      0.0  21.647354        0.0  ...   
               4                      0.0  21.647354        0.0  ...   

                            id_probabilities  vertices_indexes  vertices_x  \
entry subentry subsubentry                                                   
0     0        0                5.873783e-02                11  -56.577885   
               1                1.897528e-03                26  -57.151443   
               2                1.228780e-08                27  -54.674431   
               3                2.040330e-08                38  -57.839684   
               4                9.393646e-01                65  -58.745544   

                            vertices_y  vertices_z  vertices_energy  \
entry subentry subsubentry                                            
0     0        0             33.757492  367.699005         6.142883   
               1             34.093864  373.148987        13.982228   
               2             34.607361  373.148987         0.264199   
               3             34.662373  378.598999        36.096531   
               4             35.104450  384.049011        28.510666   

                            vertices_correctedEnergy  \
entry subentry subsubentry                             
0     0        0                                -1.0   
               1                                -1.0   
               2                                -1.0   
               3                                -1.0   
               4                                -1.0   

                            vertices_correctedEnergyUncertainty  \
entry subentry subsubentry                                        
0     0        0                                           -1.0   
               1                                           -1.0   
               2                                           -1.0   
               3                                           -1.0   
               4                                           -1.0   

                            vertices_multiplicity  layer_cluster_seed  
entry subentry subsubentry                                             
0     0        0                              1.0                  38  
               1                              1.0                  38  
               2                              1.0                  38  
               3                              1.0                  38  
               4                              1.0                  38  

[5 rows x 34 columns]

In [37]:
cluster_indices = ['vertices_x', 'vertices_y', 'vertices_z', 'vertices_energy']
event.loc[0].loc[0][cluster_indices]

vertices_x         -56.577885
vertices_y          33.757492
vertices_z         367.699005
vertices_energy      6.142883
Name: 0, dtype: float64

In [49]:
def turn_df_to_graph(df):
    graphs = []
    N_tracksters = df.NTracksters.median().astype(int)
    for i in range(N_tracksters):
        trackster = df.loc[i]
        vertices = trackster.loc[0][cluster_indices]
        vertices = vertices.values
        vertices = vertices.astype(np.float32)
        vertices = torch.from_numpy(vertices)
        graph = torch_geometric.data.Data(x=vertices)
        graphs.append(graph)
    return graphs



In [158]:
def calculate_edges(df):
    df = df[cluster_indices]
    df = df.reset_index()
    edges = []
    starts = []
    stops = []
    N_tracksters = df.NTracksters.median().astype(int)
    for i in range(N_tracksters):
        print(f"trackster {i}")
        trackster_starts = []
        trackster_stops = []
        z_values = df.loc[i]['vertices_z'].unique()
        nz = len(z_values)
        for i, z in enumerate(z_values):
            layer = df.loc[i][(df.loc[i]['vertices_z'] == z)]
            print(layer)
            pre_layer = df.loc[i]['vertices_z'] == z_values[i-1]
            for l in layer:
                for k in layer:
                    trackster_starts.append(l.subsubentry)
                    trackster_stops.append(k.subsubentry)
                for k in pre_layer:
                    trackster_starts.append(l.subsubentry)
                    trackster_stops.append(k.subsubentry)
        starts.append(trackster_starts)
        stops.append(trackster_stops)
    return starts, stops

In [160]:
def calculate_edges(df):
    starts = []
    stops = []
    N_tracksters = df.NTracksters.median().astype(int)
    for t in range(N_tracksters):
        if t > 0: break
        trackster = df.loc[t].reset_index()
        trackster_starts = []
        trackster_stops = []
        df = event[cluster_indices]
        z_values = df.loc[t]['vertices_z'].unique()
        for i, z in enumerate(z_values):
            layer = trackster.where(trackster.vertices_z == z_values[i]).dropna()['subsubentry'].values
            pre_layer = trackster.where(trackster.vertices_z == z_values[i-1]).dropna()['subsubentry'].values
            print(f"layer {i}: layer: {layer}")
            print(f"layer {i}: pre_layer: {pre_layer}")
            for l in layer:
                for k in layer:
                    trackster_starts.append(l)
                    trackster_stops.append(k)
                for k in pre_layer:
                    trackster_starts.append(l)
                    trackster_stops.append(k)

        starts.append(trackster_starts)
        stops.append(trackster_stops)
    return starts, stops